# OPIM510: Class 10 - Window Functions

## Window Functions

Defined by [Postgres](https://www.postgresql.org/docs/9.1/tutorial-window.html) as:

> A *window function* performs a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row --- the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result.

### Examples of Window Function use cases

- Ranking (or ranking within groups) based on a metric
- Running totals (or running totals within groups) of a metric
- Lead/lag calculations (i.e. Looking forward or backwards by *n* rows to perform a calculation on the current row)

## Prepare our environment

In [4]:
import polars as pl
from datetime import datetime, date

## Load our dataset

In [11]:
# Read the Superstore CSV file
sales = pl.read_csv('https://raw.githubusercontent.com/yajasarora/Superstore-Sales-Analysis-with-Tableau/refs/heads/master/Superstore%20sales%20dataset.csv')

# Convert 'Order Date' to datetime
sales = sales.with_columns(
    pl.col("Order Date").str.strptime(pl.Date, "%d/%m/%Y").alias("Order Date")
)

sales

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
i64,str,date,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,f64,i64,f64,f64
1,"""CA-2016-152156""",2016-11-08,"""11/11/2016""","""Second Class""","""CG-12520""","""Claire Gute""","""Consumer""","""United States""","""Henderson""","""Kentucky""",42420,"""South""","""FUR-BO-10001798""","""Furniture""","""Bookcases""","""Bush Somerset Collection Bookc…",261.96,2,0.0,41.9136
2,"""CA-2016-152156""",2016-11-08,"""11/11/2016""","""Second Class""","""CG-12520""","""Claire Gute""","""Consumer""","""United States""","""Henderson""","""Kentucky""",42420,"""South""","""FUR-CH-10000454""","""Furniture""","""Chairs""","""Hon Deluxe Fabric Upholstered …",731.94,3,0.0,219.582
3,"""CA-2016-138688""",2016-06-12,"""16/6/2016""","""Second Class""","""DV-13045""","""Darrin Van Huff""","""Corporate""","""United States""","""Los Angeles""","""California""",90036,"""West""","""OFF-LA-10000240""","""Office Supplies""","""Labels""","""Self-Adhesive Address Labels f…",14.62,2,0.0,6.8714
4,"""US-2015-108966""",2015-10-11,"""18/10/2015""","""Standard Class""","""SO-20335""","""Sean O'Donnell""","""Consumer""","""United States""","""Fort Lauderdale""","""Florida""",33311,"""South""","""FUR-TA-10000577""","""Furniture""","""Tables""","""Bretford CR4500 Series Slim Re…",957.5775,5,0.45,-383.031
5,"""US-2015-108966""",2015-10-11,"""18/10/2015""","""Standard Class""","""SO-20335""","""Sean O'Donnell""","""Consumer""","""United States""","""Fort Lauderdale""","""Florida""",33311,"""South""","""OFF-ST-10000760""","""Office Supplies""","""Storage""","""Eldon Fold 'N Roll Cart System""",22.368,2,0.2,2.5164
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9990,"""CA-2014-110422""",2014-01-21,"""23/1/2014""","""Second Class""","""TB-21400""","""Tom Boeckenhauer""","""Consumer""","""United States""","""Miami""","""Florida""",33180,"""South""","""FUR-FU-10001889""","""Furniture""","""Furnishings""","""Ultra Door Pull Handle""",25.248,3,0.2,4.1028
9991,"""CA-2017-121258""",2017-02-26,"""3/3/2017""","""Standard Class""","""DB-13060""","""Dave Brooks""","""Consumer""","""United States""","""Costa Mesa""","""California""",92627,"""West""","""FUR-FU-10000747""","""Furniture""","""Furnishings""","""Tenex B1-RE Series Chair Mats …",91.96,2,0.0,15.6332
9992,"""CA-2017-121258""",2017-02-26,"""3/3/2017""","""Standard Class""","""DB-13060""","""Dave Brooks""","""Consumer""","""United States""","""Costa Mesa""","""California""",92627,"""West""","""TEC-PH-10003645""","""Technology""","""Phones""","""Aastra 57i VoIP phone""",258.576,2,0.2,19.3932


## Using window functions to compute ranking

**Business Question:** Who are our top 10 most `Profit`able `Customer Name`s?

Key Points:
- We need to aggregate `Profit` for each `Customer Name`
- We need to rank the `Customer Name` groups on total `Profit`, then filter for only the top 10

In [6]:
top_customers = (
    sales
    .group_by("Customer Name")
    .agg(
        pl.col("Profit").sum().alias("total_profit")
    )
    .with_columns(
        pl.col("total_profit").rank(method="min", descending=True).alias("profit_rank")
    )
    .filter(pl.col("profit_rank") < 11)
    .sort("profit_rank")
)

top_customers

Customer Name,total_profit,profit_rank
str,f64,u32
"""Tamara Chand""",8981.3239,1
"""Raymond Buch""",6976.0959,2
"""Sanjit Chand""",5757.4119,3
"""Hunter Lopez""",5622.4292,4
"""Adrian Barton""",5444.8055,5
"""Tom Ashbrook""",4703.7883,6
"""Christopher Martinez""",3899.8904,7
"""Keith Dawkins""",3038.6254,8
"""Andy Reiter""",2884.6208,9


## Using window function within groups

**Business Question:** Who are the most `Profit`able `Customer Names` for each `Region`?

Key Points:
- We need to aggregate `Profit` for each combination of `Customer Name` and `Region`
- We need to rank the `Customer Name` groups on total `Profit`, then filter for only the top 10 FOR EACH `Region`

In [7]:
top_customers_region = (
    sales
    .group_by(["Customer Name", "Region"])
    .agg(
        pl.col("Profit").sum().alias("total_profit")
    )
    .with_columns(
        pl.col("total_profit")
        .rank(method="min", descending=True)
        .over("Region")
        .alias("profit_rank")
    )
    .filter(pl.col("profit_rank") < 11)
    .sort(["Region", "profit_rank"])
)

print(f"Shape: {top_customers_region.shape}")
top_customers_region.head(20)

Shape: (40, 4)


Customer Name,Region,total_profit,profit_rank
str,str,f64,u32
"""Tamara Chand""","""Central""",8745.0635,1
"""Adrian Barton""","""Central""",5362.6135,2
"""Sanjit Chand""","""Central""",4668.6935,3
"""Andy Reiter""","""Central""",2602.0939,4
"""Harry Marie""","""Central""",1996.8809,5
…,…,…,…
"""Karen Daniels""","""East""",2283.0463,6
"""Nathan Mautz""","""East""",2247.1904,7
"""Tom Boeckenhauer""","""East""",2239.9872,8


## Cumulative (running) totals

**Business Question:** Show the running total of `Sales` for the days (using by `Order Date`) leading up to 2016-11-08.

Key Points:
- Need to figure out how to get a running total of `Sales`
- Must order the data in ascending order by `Order Date`

In [12]:
running_sales = (
    sales
    .group_by("Order Date")
    .agg(
        pl.col("Sales").sum().alias("tot_sales")
    )
    .sort("Order Date")
    .with_columns(
        pl.col("tot_sales").cum_sum().alias("running_total")
    )
    .filter(pl.col("Order Date") < datetime(2016, 11, 8))
)

print(f"Shape: {running_sales.shape}")
running_sales.tail(10)

Shape: (864, 3)


Order Date,tot_sales,running_total
date,f64,f64
2016-10-28,2899.021,1.3835e6
2016-10-29,131.154,1.3837e6
2016-10-30,159.002,1.3838e6
2016-10-31,3750.499,1.3876e6
2016-11-01,453.374,1.3880e6
2016-11-03,3802.796,1.3918e6
2016-11-04,3888.818,1.3957e6
2016-11-05,4266.8988,1.4000e6
2016-11-06,773.256,1.4008e6


## Quick Summarization

**Business Question:** What is the the total amount of both `Sales` and `Profit` for each month of `Order Date`? (When computing months, each combination of year and month counts as a unique month.)

Key points:
- Need to figure out how to parse out only the year/months from the `Order Date` column
- We can use Polars' datetime functionality to truncate dates to month boundaries
- FOR EACH usually means we are grouping by something. In this case we have to figure out how to get year/month combinations from the `Order Date` field
- Create sums for both `Sales` and `Profit`

### Date truncation examples in Polars

In [13]:
# Examples of date truncation in Polars
example_date = datetime(2023, 11, 7, 12, 31, 13)
df_example = pl.DataFrame({"datetime": [example_date]})

print("Original datetime:", example_date)
print("\nTruncated to different units:")
print(df_example.with_columns([
    pl.col("datetime").dt.truncate("1y").alias("year"),
    pl.col("datetime").dt.truncate("1mo").alias("month"),
    pl.col("datetime").dt.truncate("1d").alias("day"),
    pl.col("datetime").dt.truncate("1h").alias("hour"),
    pl.col("datetime").dt.truncate("1m").alias("minute"),
]))

Original datetime: 2023-11-07 12:31:13

Truncated to different units:
shape: (1, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ datetime       ┆ year           ┆ month          ┆ day           ┆ hour          ┆ minute        │
│ ---            ┆ ---            ┆ ---            ┆ ---           ┆ ---           ┆ ---           │
│ datetime[μs]   ┆ datetime[μs]   ┆ datetime[μs]   ┆ datetime[μs]  ┆ datetime[μs]  ┆ datetime[μs]  │
╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ 2023-11-07     ┆ 2023-01-01     ┆ 2023-11-01     ┆ 2023-11-07    ┆ 2023-11-07    ┆ 2023-11-07    │
│ 12:31:13       ┆ 00:00:00       ┆ 00:00:00       ┆ 00:00:00      ┆ 12:00:00      ┆ 12:31:00      │
└────────────────┴────────────────┴────────────────┴───────────────┴───────────────┴───────────────┘


In [14]:
monthly_sales = (
    sales
    .with_columns(
        pl.col("Order Date").dt.truncate("1mo").alias("month")
    )
    .group_by("month")
    .agg([
        pl.col("Sales").sum().alias("tot_sales"),
        pl.col("Profit").sum().alias("tot_profit")
    ])
    .sort("month")
)

print(f"Shape: {monthly_sales.shape}")
monthly_sales.head(12)

Shape: (48, 3)


month,tot_sales,tot_profit
date,f64,f64
2014-01-01,14236.895,2450.1907
2014-02-01,4519.892,862.3084
2014-03-01,55691.009,498.7299
2014-04-01,28295.345,3488.8352
2014-05-01,23648.287,2738.7096
…,…,…
2014-08-01,27909.4685,5318.105
2014-09-01,81777.3508,8328.0994
2014-10-01,31453.393,3448.2573


## Going one level further

**Business Question:** Using the data frame from the previous question: between which months was the greatest jump in sales?

Key Points:
- Using the previous data frame
- Need to figure out the difference between `tot_sales` from month to month

In [16]:
delta_sales = (
    monthly_sales
    .with_columns(
        (pl.col("tot_sales") - pl.col("tot_sales").shift(1)).alias("month_sales_delta")
    )
)

print("\nMonth with greatest sales increase:")
print(delta_sales.sort("month_sales_delta", descending=True, nulls_last=True).head(1))

print("\nAll monthly deltas:")
delta_sales.head(12)


Month with greatest sales increase:
shape: (1, 4)
┌────────────┬────────────┬────────────┬───────────────────┐
│ month      ┆ tot_sales  ┆ tot_profit ┆ month_sales_delta │
│ ---        ┆ ---        ┆ ---        ┆ ---               │
│ date       ┆ f64        ┆ f64        ┆ f64               │
╞════════════╪════════════╪════════════╪═══════════════════╡
│ 2014-09-01 ┆ 81777.3508 ┆ 8328.0994  ┆ 53867.8823        │
└────────────┴────────────┴────────────┴───────────────────┘

All monthly deltas:


month,tot_sales,tot_profit,month_sales_delta
date,f64,f64,f64
2014-01-01,14236.895,2450.1907,null
2014-02-01,4519.892,862.3084,-9717.003
2014-03-01,55691.009,498.7299,51171.117
2014-04-01,28295.345,3488.8352,-27395.664
2014-05-01,23648.287,2738.7096,-4647.058
…,…,…,…
2014-08-01,27909.4685,5318.105,-6036.9245
2014-09-01,81777.3508,8328.0994,53867.8823
2014-10-01,31453.393,3448.2573,-50323.9578


## Same dataset but now finding the `Profit` monthly delta

**Business Question:** Using the data frame from the previous question: between which months was the greatest jump in profit?

Key Points:
- Using the previous data frame
- Need to figure out the difference between `tot_profit` from month to month

In [17]:
delta_sales = (
    delta_sales
    .with_columns(
        (pl.col("tot_profit") - pl.col("tot_profit").shift(1)).alias("month_profit_delta")
    )
)

print("\nMonth with greatest profit increase:")
print(delta_sales.sort("month_profit_delta", descending=True, nulls_last=True).head(1))

print("\nAll monthly deltas:")
delta_sales.head(12)


Month with greatest profit increase:
shape: (1, 5)
┌────────────┬───────────┬────────────┬───────────────────┬────────────────────┐
│ month      ┆ tot_sales ┆ tot_profit ┆ month_sales_delta ┆ month_profit_delta │
│ ---        ┆ ---       ┆ ---        ┆ ---               ┆ ---                │
│ date       ┆ f64       ┆ f64        ┆ f64               ┆ f64                │
╞════════════╪═══════════╪════════════╪═══════════════════╪════════════════════╡
│ 2016-12-01 ┆ 96999.043 ┆ 17885.3093 ┆ 17587.0772        ┆ 13873.9018         │
└────────────┴───────────┴────────────┴───────────────────┴────────────────────┘

All monthly deltas:


month,tot_sales,tot_profit,month_sales_delta,month_profit_delta
date,f64,f64,f64,f64
2014-01-01,14236.895,2450.1907,null,null
2014-02-01,4519.892,862.3084,-9717.003,-1587.8823
2014-03-01,55691.009,498.7299,51171.117,-363.5785
2014-04-01,28295.345,3488.8352,-27395.664,2990.1053
2014-05-01,23648.287,2738.7096,-4647.058,-750.1256
…,…,…,…,…
2014-08-01,27909.4685,5318.105,-6036.9245,6159.5876
2014-09-01,81777.3508,8328.0994,53867.8823,3009.9944
2014-10-01,31453.393,3448.2573,-50323.9578,-4879.8421


## Additional Window Function Examples

### Lead function example
Let's look at the next month's sales alongside current month

In [ ]:
# Using shift with negative value for lead
lead_example = (
    monthly_sales
    .with_columns([
        pl.col("tot_sales").shift(-1).alias("next_month_sales"),
        pl.col("tot_sales").shift(1).alias("prev_month_sales")
    ])
)

print("Lead/Lag example:")
lead_example.head(10)

### Rolling window calculations
Calculate 3-month moving average of sales

In [18]:
rolling_example = (
    monthly_sales
    .sort("month")
    .with_columns(
        pl.col("tot_sales").rolling_mean(window_size=3).alias("3_month_avg_sales")
    )
)

print("Rolling 3-month average:")
rolling_example.head(12)

Rolling 3-month average:


month,tot_sales,tot_profit,3_month_avg_sales
date,f64,f64,f64
2014-01-01,14236.895,2450.1907,null
2014-02-01,4519.892,862.3084,null
2014-03-01,55691.009,498.7299,24815.932
2014-04-01,28295.345,3488.8352,29502.082
2014-05-01,23648.287,2738.7096,35878.213667
…,…,…,…
2014-08-01,27909.4685,5318.105,32150.3297
2014-09-01,81777.3508,8328.0994,47877.737433
2014-10-01,31453.393,3448.2573,47046.737433
